# PRM Evaluation

Based on the paper, the evaluation is using N sampling method. So I need to implement the N sampling method to get the best solution
Generator: Generate the entire solution for N times
Critic: Evaluate the solution for N times. 

We can test it with the grading method from the dataset. Also we can directly select solution from the dataset "is_solution" column
The output of the model is in the format of 


'### [Solution]'

So technically I can just extract the end of the output and use the grader to verify the solution

In [ ]:
model_answer =  " 320,000"

gt_answer = "320,000"

True

## Data (scrapped since it's PRM and too complex)

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", padding_side='right', use_fast = False)

EOS_TOKEN = tokenizer.eos_token  # End-of-Sequence token
prompt = """
### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    texts = []
    final_answer = []
    
    for instruction, next_response, answer in zip(examples['instruction'], examples['next_response'], examples['answer']):
        # Combine all responses and the next response into a single string with newline separation
        if answer is not None:
            final_answer.append(answer)
        else:
            cleaned_answer = next_response.split('# Answer')[1]
            final_answer.append(cleaned_answer)
        
        # Format the text with the prompt template
        text = prompt.format(instruction, '')
        texts.append(text)

    # Tokenize all texts at once using the tokenizer
    # model_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    
    return {'input_text': texts, 'final_answer': final_answer}


/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
from datasets import load_dataset

# Load and preprocess the dataset
dataset = load_dataset("Birchlabs/openai-prm800k-stepwise-critic", split='test')
# dataset = dataset.filter(lambda x: x['answer'] is not None)  # Filter entries without ratings
dataset = dataset.filter(lambda x: x['is_solution'] is True)  # Filter entries without ratings

dataset = dataset.map(formatting_prompts_func, batched=True)  # Apply the preprocessing function


## Generator
Generator generate the solution for N times. The critic evaluate the solution for N times. The critic will output the score of the solution. The generator will output the solution with the highest score

In [20]:
# from unsloth import FastLanguageModel
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM

import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


generator = LlamaForCausalLM.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")
tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", padding_side='right', use_fast = False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda

generator.to(device)

#set generator to inference mode
generator.eval()



LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  

In [23]:
#dataset preparation
input_text = dataset['input_text']
inputs = tokenizer(input_text, return_tensors='pt',padding="max_length", truncation=True, max_length=512).to(device)

In [24]:
# prompt = """
# ### Input:
# {}

# ### Response:
# {}"""

# inputs = tokenizer(
# [
#     prompt.format(
#         "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?", # Question
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

outputs = generator.generate(**inputs, max_new_tokens = 256, use_cache = True, do_sample=True, temperature=0.5, top_k=40)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [27]:
answers = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
def generate_answers(input_text, generator, tokenizer):
    inputs = tokenizer(input_text, return_tensors='pt',padding="max_length", truncation=True, max_length=512).to(device)
    outputs = generator.generate(**inputs, max_new_tokens = 256, use_cache = True, do_sample=True, temperature=0.5, top_k=40)
    answers = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answers

In [28]:
#save answers to a local file


"\n### Input:\n('Three pencils and a jumbo eraser cost $\\\\$1.24$. Five pencils and a jumbo eraser cost $\\\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?',)\n\n### Response:\nat $1.24 per pencil, 3 pencils cost 3*1.24 = $<<3*1.24=3.68>>3.68\nat $1.82 per pencil, 5 pencils cost 5*1.82 = $<<5*1.82=9.64>>9.64\nIn cents, the cost of a pencil is $3.68 + $9.64 = $<<3.68+9.64=13.32>>13.32\n#### 1332"

In [29]:
dataset['answer'][0]

'29'

In [35]:
def save_strings_to_file(strings, file_path):
    """
    Saves a list of strings to a file.
    
    Args:
        strings (list): The list of strings to be saved.
        file_path (str): The path of the file to save the strings.
    """
    with open(file_path, "w") as file:
        for string in strings:
            # Replace newline characters with a placeholder
            string = string.replace("\n", "\\n")
            file.write(string + "\n")
    
    print("Strings saved to", file_path)


def read_strings_from_file(file_path):
    """
    Reads a list of strings from a file.
    
    Args:
        file_path (str): The path of the file to read the strings from.
        
    Returns:
        list: The list of strings read from the file.
    """
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    strings = [line.strip().replace("\\n", "\n") for line in lines]
    
    return strings


In [36]:
save_strings_to_file(answers, "test1.txt")

Strings saved to test1.txt


In [37]:
answers2 = read_strings_from_file("test1.txt")

In [39]:
answers2[0]

"\n### Input:\n('Three pencils and a jumbo eraser cost $\\\\$1.24$. Five pencils and a jumbo eraser cost $\\\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?',)\n\n### Response:\nat $1.24 per pencil, 3 pencils cost 3*1.24 = $<<3*1.24=3.68>>3.68\nat $1.82 per pencil, 5 pencils cost 5*1.82 = $<<5*1.82=9.64>>9.64\nIn cents, the cost of a pencil is $3.68 + $9.64 = $<<3.68+9.64=13.32>>13.32\n#### 1332"

## Critic
First I need to test whether critic actually work for the wrong solutions

In [40]:
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM
import torch
critic = LlamaForSequenceClassification.from_pretrained("/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic")
critic.eval()

critic_tokenizer = AutoTokenizer.from_pretrained("/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic", padding_side='right', use_fast= False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda
critic.to(device)


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [ ]:
prompt = """
### Input:
{}

### Response:
{}"""

Question = "How many positive two-digit integers leave a remainder of 2 when divided by 8?"
Answer = [ "So if a number leaves a remainder of 2 when divided by 8, it's of the form 8n+2.", "So we want to know the number of positive two-digit integers of the form 8n+2.", "I think we should just plug in numbers and see what happens.", "Ok let's start with n=1.", "8*1+2=10 which is a two-digit integer.", "Let's try n=2.", "8*2+2=18 which is also a two-digit integer." ]
wrong_answer = "And if we keep going we'll see that all the numbers of the form 8n+2 are two-digit integers."

combined_responses = ""
for i in range(len(Answer)):
    combined_responses += Answer[i] + "\n"
    
combined_wrong_responses = combined_responses + wrong_answer

input_text = prompt.format(Question, Answer)
inputs = critic_tokenizer(combined_wrong_responses, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = critic(**inputs)
print(outputs.logits)

tensor([[ 0.4651, -1.4561, -0.1776]], device='cuda:0')


In [ ]:
#take the softmax of the logits
import torch.nn.functional as F
probabilities = F.softmax(outputs.logits, dim=1)
print(probabilities)


tensor([[0.5980, 0.0876, 0.3145]], device='cuda:0')


In [ ]:
result = '### Input:\nWHow many seconds are in 7.8 minutes?\n\n### Response:\n'
responses = [ "7.8 minutes is the same as 7 minutes and 0.8 minutes.", "Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes.", "And since there are 60 seconds in a minute, then there are 60 * 0.8 = 48 seconds in 0.8 minutes.", "So, in total, there are 420 + 48 = 468 seconds in 7.8 minutes." ]


In [56]:
answers[0]

"\n### Input:\n('Three pencils and a jumbo eraser cost $\\\\$1.24$. Five pencils and a jumbo eraser cost $\\\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?',)\n\n### Response:\nat $1.24 per pencil, 3 pencils cost 3*1.24 = $<<3*1.24=3.68>>3.68\nat $1.82 per pencil, 5 pencils cost 5*1.82 = $<<5*1.82=9.64>>9.64\nIn cents, the cost of a pencil is $3.68 + $9.64 = $<<3.68+9.64=13.32>>13.32\n#### 1332"

In [61]:
import torch.nn.functional as F
answers_prob = []

with torch.no_grad():
    for answer in answers:
        result = answer.split('### Response:\n')[0] + '### Response:\n'
        responses = answer.split('### Response:\n')[1].split('\n')
        correct_probability = 1
        incorrect_probability = 1
        for response in responses:
            result += response + "\n"
            inputs = critic_tokenizer(result, return_tensors="pt").to("cuda")
            outputs = critic(**inputs)
            #take the softmax of the logits
            probabilities = F.softmax(outputs.logits, dim=1)
            correct_probability *= (probabilities[0][2]+ probabilities[0][1])
        answers_prob.append(correct_probability.to("cpu").numpy())


In [63]:
answers_prob = [float(i) for i in answers_prob]

In [73]:
dataset['answer'][-18]

'29'

In [72]:
print(answers[-18])


### Input:
('Three pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?',)

### Response:
  Three pencils and a jumbo eraser cost $\\$1.24$, so 1.24*3 = $\\$3.68
  Five pencils and a jumbo eraser cost $\\$1.82$, so 1.82*5 = $\\$9.64
  The cost of a pencil is 3+9 = $\\$12.
#### 12


In [69]:
answers_prob[-18]

0.5060761570930481

In [51]:
# Get the predicted labels
predicted_labels = torch.argmax(probabilities, dim=1)

In [52]:
predicted_labels

tensor([2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2,
        0, 0, 0, 0, 0, 0], device='cuda:0')

In [ ]:
result = ouput_text[0].split('### Response:\n')[0] + '### Response:\n'
responses = ouput_text[0].split('### Response:\n')[1].split('\n')

In [ ]:
correct_probability = 1
incorrect_probability = 1
neutral_probability = 1

with torch.no_grad():
    for response in responses:
        result += response + "\n"
        inputs = critic_tokenizer(result, return_tensors="pt").to("cuda")
        outputs = critic(**inputs)
        #take the softmax of the logits
        import torch.nn.functional as F
        probabilities = F.softmax(outputs.logits, dim=1)
        print(probabilities)
        correct_probability *= (probabilities[0][2]+ probabilities[0][1])
        incorrect_probability *= probabilities[0][0]
        # neutral_probability *= 
    

    

tensor([[0.1941, 0.0384, 0.7675]], device='cuda:0')
tensor([[0.0710, 0.0134, 0.9156]], device='cuda:0')
tensor([[0.0393, 0.0118, 0.9489]], device='cuda:0')
tensor([[0.0174, 0.0075, 0.9751]], device='cuda:0')


In [ ]:
print(correct_probability)
print(incorrect_probability)
# print(neutral_probability)

tensor(0.7068, device='cuda:0')
tensor(9.4226e-06, device='cuda:0')


In [ ]:
responses

['First find how many bolts of blue fiber the robe takes: 2 bolts of blue fiber * 2/2 = <<2*2=4>>4 bolts',
 'Then find how many bolts of white fiber the robe takes: 4 bolts of white fiber / 2 = <<4/2=2>>2 bolts',
 'Then add the two bolt counts to find the total number of bolts: 2 bolts of blue fiber + 2 bolts of white fiber = <<2+2=4>>4 bolts',
 '#### 4']

In [ ]:
inputs = critic_tokenizer(result, return_tensors="pt").to("cuda")
outputs = critic(**inputs)
#take the softmax of the logits
import torch.nn.functional as F
probabilities = F.softmax(outputs.logits, dim=1)
print(probabilities)


tensor([[0.4339, 0.2652, 0.3009]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [ ]:
probabilities_total = probabilities_total* probabilities
probabilities_total

tensor([[1.6243e-02, 6.9465e-06, 9.4159e-02]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [ ]:
probabilities_total

tensor([[1.3050e-02, 1.7264e-06, 1.1865e-01]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [ ]:
output = ouput_text[0].split("<s> \n")[1].split("</s>")[0]

In [ ]:
output

'### Input:\nA robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?\n\n### Response:\nFirst find the total number of blue fiber: 2 bolts * 2 = <<2*2=4>>4 bolts\nThen find the total number of white fiber: 2 bolts / 2 = <<2/2=1>>1 bolt\nThen add the blue and white fiber to find the total number of fiber: 4 bolts + 1 bolt = <<4+1=5>>5 bolts\n#### 5'

In [ ]:
#turn outputs into tokens
answer = critic_tokenizer(output, return_tensors="pt").to("cuda")

In [ ]:
answer

{'input_ids': tensor([[    1,   835, 10567, 29901,    13, 29909,   696,   915,  4893, 29871,
         29906, 15772,  1372,   310,  7254,  5713,   495,   322,  4203,   393,
          1568,  4796,  5713,   495, 29889,  1128,  1784, 15772,  1372,   297,
          3001,   947,   372,  2125, 29973,    13,    13,  2277, 29937, 13291,
         29901,    13,  6730,  1284,   278,  3001,  1353,   310,  7254,  5713,
           495, 29901, 29871, 29906, 15772,  1372,   334, 29871, 29906,   353,
          3532, 29906, 29930, 29906, 29922, 29946,  6778, 29946, 15772,  1372,
            13, 11760,  1284,   278,  3001,  1353,   310,  4796,  5713,   495,
         29901, 29871, 29906, 15772,  1372,   847, 29871, 29906,   353,  3532,
         29906, 29914, 29906, 29922, 29896,  6778, 29896,   289, 14339,    13,
         11760,   788,   278,  7254,   322,  4796,  5713,   495,   304,  1284,
           278,  3001,  1353,   310,  5713,   495, 29901, 29871, 29946, 15772,
          1372,   718, 29871, 29896,  

In [ ]:
outputs = critic(**answer)
print(outputs.logits)

AttributeError: 'LlamaModel' object has no attribute '_has_no_labels'

# Data

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", padding_side='right', use_fast = False)

EOS_TOKEN = tokenizer.eos_token  # End-of-Sequence token
prompt = """
### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    texts = []
    final_answer = []
    
    for instruction, answer in zip(examples['question'], examples['answer']):
        # Combine all responses and the next response into a single string with newline separation
        extracted_answer = answer.split('### ')[1]
        final_answer.append(extracted_answer)
        
        # Format the text with the prompt template
        text = prompt.format(instruction, '')
        texts.append(text)

    # Tokenize all texts at once using the tokenizer
    # model_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    
    return {'input_text': texts, 'final_answer': final_answer}


/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
from datasets import load_dataset

# Load and preprocess the dataset
dataset = load_dataset("gsm8k", 'main', split='test')
dataset = dataset.map(formatting_prompts_func, batched=True)  # Apply the preprocessing function

len(dataset)

1319

# Batch Generate Answers

In [4]:
# from unsloth import FastLanguageModel
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM

import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


generator = LlamaForCausalLM.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")
tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", padding_side='right', use_fast = False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda

generator.to(device)

#set generator to inference mode
generator.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  

In [5]:
#dataset preparation
input_text = dataset['input_text']
# inputs = tokenizer(input_text, return_tensors='pt',padding="max_length", truncation=True, max_length=512).to(device)

In [6]:
import os
import json
import re

def generate_answers(input_text, generator, tokenizer, output_dir, n_answers=100, batch_size=128):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    
    # Get the existing JSON file numbers in the directory
    existing_files = os.listdir(output_dir)
    file_numbers = []
    for file_name in existing_files:
        match = re.match(r"generated_answers_(\d+)\.json", file_name)
        if match:
            file_numbers.append(int(match.group(1)))
    
    # Start generating new answers from the next available number
    start_number = max(file_numbers, default=0) + 1
    
    for n in range(start_number, start_number + n_answers):
        all_answers = []
        for i in range(0, len(input_text), batch_size):
            batch_inputs = input_text[i:i+batch_size]
            batch_inputs = tokenizer(batch_inputs, return_tensors='pt', padding="max_length", truncation=True, max_length=512).to(device)
            outputs = generator.generate(
                **batch_inputs,
                max_new_tokens=256,
                use_cache=True,
                do_sample=True,
                temperature=0.5,
                top_k=40
            )
            answers = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            all_answers.extend(answers)
        
        # Generate a unique file name for each set of n_answers
        file_name = f"generated_answers_{n}.json"
        file_path = os.path.join(output_dir, file_name)
        
        # Save the answers to a JSON file for each set of n_answers in the specified directory
        with open(file_path, 'w') as file:
            json.dump(all_answers, file)
        
        print(f"Generated {len(all_answers)} answers for set {n}.")
    
    return all_answers

In [7]:
output_directory = "generated_answers"

with torch.no_grad():
    answers = generate_answers(input_text, generator, tokenizer, output_directory, n_answers=100)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Generated 1319 answers for set 1.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Generated 1319 answers for set 2.


...

Generated 1319 answers for set 98.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Generated 1319 answers for set 99.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Generated 1319 answers for set 100.


# Majority Voting Evaluation

In [49]:
import os
import json
import re
from collections import Counter
import sys
sys.path.append('Data/prm800k/prm800k')
from grading import grader

def load_json_answers(directory):
    json_files = [file for file in os.listdir(directory) if file.endswith(".json")]
    all_answers = []
    for file in json_files:
        file_path = os.path.join(directory, file)
        with open(file_path, "r") as f:
            answers = json.load(f)
            all_answers.append(answers)
    return all_answers

def extract_answers(all_answers):
    extracted_answers = [[] for _ in range(len(all_answers[0]))]
    for answers in all_answers:
        for i, answer in enumerate(answers):
            match = re.search(r"####\s*(.*)", answer)
            if match:
                extracted_answers[i].append(match.group(1).strip())
            else:
                extracted_answers[i].append("")
    return extracted_answers

def majority_vote(extracted_answers):
    majority_answers = []
    for question_answers in extracted_answers:
        valid_answers = [answer for answer in question_answers if answer and answer.strip()]
        if valid_answers:
            counter = Counter(valid_answers)
            majority_answer = counter.most_common(1)[0][0]
        else:
            majority_answer = ""
        majority_answers.append(majority_answer)
    return majority_answers

def compare_with_ground_truth(majority_answers, ground_truth_answers):
    correct_count = 0
    for majority_answer, ground_truth_answer in zip(majority_answers, ground_truth_answers):
        if grader.grade_answer(majority_answer, ground_truth_answer):
            correct_count += 1
    accuracy = correct_count / len(ground_truth_answers)
    return accuracy


In [50]:

# Usage example
json_directory = "generated_answers"
all_answers = load_json_answers(json_directory)
extracted_answers = extract_answers(all_answers)
majority_answers = majority_vote(extracted_answers)


In [51]:
accuracy = compare_with_ground_truth(majority_answers, dataset['final_answer'])

# print("Majority Answers:")
# for answer in majority_answers:
#     print(answer)

print(f"\nAccuracy: {accuracy:.2f}")


Accuracy: 0.13


# PRM Evaluation

In [9]:
import os
import json
import re
import torch
import torch.nn.functional as F
from tqdm import tqdm
import sys
sys.path.append('Data/prm800k/prm800k')
from grading import grader

def load_json_answers(directory):
    json_files = [file for file in os.listdir(directory) if file.endswith(".json")]
    all_answers = []
    for file in json_files:
        file_path = os.path.join(directory, file)
        with open(file_path, "r") as f:
            answers = json.load(f)
            all_answers.append(answers)
    return all_answers

def extract_answers(all_answers):
    extracted_answers = [[] for _ in range(len(all_answers[0]))]
    for answers in all_answers:
        for i, answer in enumerate(answers):
            match = re.search(r"####\s*(.*)", answer)
            if match:
                extracted_answers[i].append(match.group(1).strip())
            else:
                extracted_answers[i].append("")
    return extracted_answers

def compute_probabilities(all_answers, critic_tokenizer, critic, batch_size=128):
    answers_prob = [[] for _ in range(len(all_answers[0]))]
    with torch.no_grad():
        for answers in tqdm(all_answers, desc="Processing answers"):
            results = []
            for answer in answers:
                result = answer.split('### Response:\n')[0] + '### Response:\n'
                responses = answer.split('### Response:\n')[1].split('\n')
                results.extend([result + response + "\n" for response in responses])
            
            correct_probabilities = []
            for i in range(0, len(results), batch_size):
                batch_results = results[i:i+batch_size]
                inputs = critic_tokenizer(batch_results, padding="max_length", truncation=True, max_length=256, return_tensors="pt").to("cuda")
                outputs = critic(**inputs)
                probabilities = F.softmax(outputs.logits, dim=1)
                correct_probability = probabilities[:, 1:]
                correct_probability = torch.sum(correct_probability, dim=1)
                correct_probabilities.extend(correct_probability.tolist())
            
            for i, answer in enumerate(answers):
                num_responses = len(answer.split('### Response:\n')[1].split('\n'))
                answer_prob = torch.tensor(correct_probabilities[i:i+num_responses]).prod().item()
                answers_prob[i].append(answer_prob)
    
    return answers_prob

def select_highest_probability_answers(extracted_answers, answers_prob):
    highest_probability_answers = []
    for i, question_answers in enumerate(extracted_answers):
        question_probs = answers_prob[i]
        if question_probs:
            max_prob_index = question_probs.index(max(question_probs))
            highest_probability_answer = question_answers[max_prob_index]
        else:
            highest_probability_answer = ""
        highest_probability_answers.append(highest_probability_answer)
    return highest_probability_answers

def compare_with_ground_truth(majority_answers, ground_truth_answers):
    correct_count = 0
    for majority_answer, ground_truth_answer in zip(majority_answers, ground_truth_answers):
        if grader.grade_answer(majority_answer, ground_truth_answer):
            correct_count += 1
    accuracy = correct_count / len(ground_truth_answers)
    return accuracy


In [67]:
# import torch.nn.functional as F
# answers_prob = []

# with torch.no_grad():
#     for answer in answers:
#         result = answer.split('### Response:\n')[0] + '### Response:\n'
#         responses = answer.split('### Response:\n')[1].split('\n')
#         correct_probability = 1
#         incorrect_probability = 1
#         results = []
#         for response in responses:
#             result += response + "\n"
#             results.append(result)
#         inputs = critic_tokenizer(results, padding="max_length", truncation=True, max_length=256, return_tensors="pt").to("cuda")
#         outputs = critic(**inputs)
#         #take the softmax of the logits
#         probabilities = F.softmax(outputs.logits, dim=1)
#         correct_probability = probabilities[:, 1:]
#         correct_probability = torch.sum(correct_probability, dim=1)
#         correct_probability = torch.prod(correct_probability)
#         # answers_prob.append(correct_probability.to("cpu").numpy())


In [74]:
# correct_probability = 1 
# for i in probabilities:
#     correct_probability *= (i[2]+ i[1])
# print(correct_probability)

tensor(0.1915, device='cuda:0')


In [71]:
# correct_probability = probabilities[:, 1:]
# correct_probability = torch.sum(correct_probability, dim=1)
# correct_probability = torch.prod(correct_probability)

In [72]:
# correct_probability

tensor(0.1915, device='cuda:0')

In [4]:
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM
import torch

# Usage example
json_directory = "generated_answers"
critic = LlamaForSequenceClassification.from_pretrained("/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic")
critic.eval()

critic_tokenizer = AutoTokenizer.from_pretrained("/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic", padding_side='right', use_fast= False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda
critic.to(device)


all_answers = load_json_answers(json_directory)
extracted_answers = extract_answers(all_answers)
answers_prob = compute_probabilities(all_answers, critic_tokenizer, critic)
highest_probability_answers = select_highest_probability_answers(extracted_answers, answers_prob)

# print("Highest Probability Answers:")
# for answer in highest_probability_answers:
#     print(answer)

Processing answers: 100%|██████████| 100/100 [2:46:55<00:00, 100.16s/it] 


In [5]:
extracted_answers = extract_answers(all_answers)
highest_probability_answers = select_highest_probability_answers(extracted_answers, answers_prob)


In [10]:
accuracy = compare_with_ground_truth(highest_probability_answers, dataset['final_answer'])

# print("Majority Answers:")
# for answer in majority_answers:
#     print(answer)

print(f"\nAccuracy: {accuracy:.2f}")


Accuracy: 0.06


In [30]:
highest_probability_answers[2]

'45000'

In [32]:
extracted_answers[2][36]

'45000'

In [31]:
answers_prob[2].index(max(answers_prob[2]))

36